In [1]:
proceeded_ids = ['OPX_001', 'OPX_002', 'OPX_003', 'OPX_004', 'OPX_005', 'OPX_006', 'OPX_007', 'OPX_008', 'OPX_009', 'OPX_011', 
'OPX_012', 'OPX_013', 'OPX_014', 'OPX_015', 'OPX_016', 'OPX_017', 'OPX_018', 'OPX_019', 'OPX_020', 'OPX_021', 'OPX_022', 
'OPX_023', 'OPX_025', 'OPX_026', 'OPX_027', 'OPX_028', 'OPX_029', 'OPX_030']

In [5]:
cur_id = 'OPX_002'

if cur_id not in proceeded_ids:
    print("NOT")